In [ ]:
import cv2
import numpy as np
import ipywidgets
import traitlets
from IPython.display import display
from jetbot import Camera, Robot, bgr8_to_jpeg
import time
import matplotlib.pyplot as plt

In [ ]:
# Khởi tạo camera và robot cho computer vision line detection
print("Đang khởi tạo camera và robot...")

camera = Camera()
robot = Robot()

# Tạo widget hiển thị hình ảnh gốc và đã xử lý
image_widget = ipywidgets.Image(format='jpeg', width=300, height=300)
processed_widget = ipywidgets.Image(format='jpeg', width=300, height=300)

# Kết nối camera với widget gốc
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

print("✓ Camera và Robot đã sẵn sàng!")
print("✓ Widget hiển thị đã được tạo")

# Hiển thị cả hai hình ảnh
print("📷 Hình ảnh gốc và đã xử lý:")
display(ipywidgets.HBox([image_widget, processed_widget]))

In [ ]:
# Hàm xử lý hình ảnh để detect đường màu đen
def detect_black_line(image):
    """
    Phát hiện đường màu đen bằng computer vision
    Returns: (center_x, confidence) hoặc (None, 0) nếu không tìm thấy
    """
    # Chuyển sang grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Blur để giảm noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Threshold để tìm vùng tối (đường đen)
    # Giá trị thấp = đen, giá trị cao = trắng
    _, binary = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV)
    
    # Chỉ quan tâm đến nửa dưới của hình ảnh
    height, width = binary.shape
    roi_start = int(height * 0.5)  # Bắt đầu từ giữa hình
    roi = binary[roi_start:, :]
    
    # Tìm contours (đường viền)
    contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None, 0, binary
    
    # Tìm contour lớn nhất (có thể là đường)
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Kiểm tra diện tích tối thiểu
    area = cv2.contourArea(largest_contour)
    if area < 100:  # Quá nhỏ, có thể là noise
        return None, 0, binary
    
    # Tính toán center của contour
    M = cv2.moments(largest_contour)
    if M["m00"] != 0:
        center_x = int(M["m10"] / M["m00"])
        center_y = int(M["m01"] / M["m00"]) + roi_start
        
        # Normalize center_x về khoảng [-1, 1]
        normalized_x = (center_x - width/2) / (width/2)
        
        # Confidence dựa trên diện tích
        confidence = min(area / 1000, 1.0)
        
        return normalized_x, confidence, binary
    
    return None, 0, binary

def create_debug_image(original, binary, center_x=None):
    """Tạo hình ảnh debug để hiển thị"""
    # Tạo hình ảnh 3 channel từ binary
    debug_img = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
    
    # Vẽ đường trung tâm
    height, width = binary.shape
    cv2.line(debug_img, (width//2, 0), (width//2, height), (0, 255, 0), 2)
    
    # Vẽ điểm center nếu có
    if center_x is not None:
        pixel_x = int((center_x * width/2) + width/2)
        cv2.circle(debug_img, (pixel_x, height//2), 10, (0, 0, 255), -1)
        
        # Vẽ đường từ center đến điểm phát hiện
        cv2.line(debug_img, (width//2, height//2), (pixel_x, height//2), (255, 0, 0), 3)
    
    return debug_img

print("✓ Hàm computer vision đã sẵn sàng")

In [ ]:
# Test hàm computer vision với frame hiện tại
print("🧪 Test computer vision với frame hiện tại...")

# Lấy frame hiện tại từ camera
current_frame = camera.value

# Test hàm detect line
center_x, confidence, binary = detect_black_line(current_frame)

print(f"Center X: {center_x}")
print(f"Confidence: {confidence}")

# Tạo và hiển thị hình ảnh debug
debug_img = create_debug_image(current_frame, binary, center_x)

# Chuyển thành JPEG để hiển thị
_, buffer = cv2.imencode('.jpg', debug_img)
processed_widget.value = buffer.tobytes()

print("✓ Test thành công! Xem hình ảnh đã xử lý bên phải.")

## Điều khiển thông số JetBot

Các slider dưới đây giúp bạn điều chỉnh hành vi của robot:

- **Speed Gain**: Tốc độ chung của robot (0.0 - 1.0)
- **Steering Gain**: Độ mạnh của việc rẽ (0.0 - 1.0) 
- **Steering Derivative**: Bộ điều khiển PD cho steering (0.0 - 0.5)
- **Steering Bias**: Hiệu chỉnh bias trái/phải (-0.3 - 0.3)

In [ ]:
# Tạo các slider điều khiển cho computer vision
speed_gain_slider = ipywidgets.FloatSlider(
    min=0.0, max=1.0, step=0.01, value=0.15, 
    description='Speed Gain'
)

steering_gain_slider = ipywidgets.FloatSlider(
    min=0.0, max=2.0, step=0.01, value=0.5, 
    description='Steering Gain'
)

steering_dgain_slider = ipywidgets.FloatSlider(
    min=0.0, max=0.5, step=0.001, value=0.05, 
    description='Steering kD'
)

steering_bias_slider = ipywidgets.FloatSlider(
    min=-0.3, max=0.3, step=0.01, value=0.0, 
    description='Steering Bias'
)

# Threshold cho computer vision
threshold_slider = ipywidgets.IntSlider(
    min=30, max=120, step=5, value=60,
    description='Black Threshold'
)

min_area_slider = ipywidgets.IntSlider(
    min=50, max=500, step=25, value=100,
    description='Min Area'
)

print("🎛️ Các slider điều khiển:")
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)
print("🔧 Tham số computer vision:")
display(threshold_slider, min_area_slider)

In [ ]:
# Tạo slider hiển thị trạng thái robot
center_x_slider = ipywidgets.FloatSlider(
    min=-1.0, max=1.0, description='Center X', 
    disabled=True
)

confidence_slider = ipywidgets.FloatSlider(
    min=0, max=1.0, description='Confidence', 
    disabled=True
)

steering_slider = ipywidgets.FloatSlider(
    min=-1.0, max=1.0, description='Steering', 
    disabled=True
)

speed_slider = ipywidgets.FloatSlider(
    min=0, max=1.0, description='Speed', 
    disabled=True
)

print("📊 Hiển thị trạng thái robot:")
display(center_x_slider, confidence_slider)
display(steering_slider, speed_slider)

## Hàm điều khiển Computer Vision

Hàm này sử dụng OpenCV để:
1. Detect đường màu đen bằng thresholding
2. Tìm contour lớn nhất
3. Tính toán center của đường
4. Điều khiển robot theo PID controller

In [ ]:
# Biến toàn cục cho PID controller
steering = 0.0
last_steering = 0.0

def detect_black_line_dynamic(image):
    """Hàm detect line với threshold động từ slider"""
    # Chuyển sang grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Blur để giảm noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Threshold với giá trị từ slider
    _, binary = cv2.threshold(blurred, threshold_slider.value, 255, cv2.THRESH_BINARY_INV)
    
    # ROI - chỉ quan tâm nửa dưới
    height, width = binary.shape
    roi_start = int(height * 0.5)
    roi = binary[roi_start:, :]
    
    # Tìm contours
    contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None, 0, binary
    
    # Tìm contour lớn nhất
    largest_contour = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(largest_contour)
    
    # Kiểm tra diện tích tối thiểu từ slider
    if area < min_area_slider.value:
        return None, 0, binary
    
    # Tính center
    M = cv2.moments(largest_contour)
    if M["m00"] != 0:
        center_x = int(M["m10"] / M["m00"])
        normalized_x = (center_x - width/2) / (width/2)
        confidence = min(area / 1000, 1.0)
        return normalized_x, confidence, binary
    
    return None, 0, binary

def execute_cv(change):
    """Hàm điều khiển chính sử dụng computer vision"""
    global steering, last_steering
    
    try:
        # Lấy hình ảnh mới từ camera
        image = change['new']
        
        # Phát hiện đường bằng computer vision
        center_x, confidence, binary = detect_black_line_dynamic(image)
        
        # Cập nhật slider hiển thị
        center_x_slider.value = center_x if center_x is not None else 0
        confidence_slider.value = confidence
        speed_slider.value = speed_gain_slider.value
        
        # Tạo và hiển thị hình ảnh debug
        debug_img = create_debug_image(image, binary, center_x)
        _, buffer = cv2.imencode('.jpg', debug_img)
        processed_widget.value = buffer.tobytes()
        
        # Điều khiển robot nếu phát hiện được đường
        if center_x is not None and confidence > 0.1:
            # PID controller
            steering = center_x * steering_gain_slider.value + (center_x - last_steering) * steering_dgain_slider.value
            last_steering = center_x
            
            # Áp dụng bias
            steering = steering + steering_bias_slider.value
            
            # Giới hạn steering
            steering = max(min(steering, 1.0), -1.0)
            
            # Cập nhật slider
            steering_slider.value = steering
            
            # Điều khiển động cơ
            left_speed = max(min(speed_slider.value + steering, 1.0), 0.0)
            right_speed = max(min(speed_slider.value - steering, 1.0), 0.0)
            
            robot.left_motor.value = left_speed
            robot.right_motor.value = right_speed
            
        else:
            # Không phát hiện được đường - dừng robot
            steering_slider.value = 0
            robot.stop()
        
    except Exception as e:
        print(f"Lỗi trong quá trình điều khiển: {e}")
        robot.stop()

print("✓ Hàm computer vision điều khiển đã sẵn sàng")
print("⚠️ Chú ý: Robot sẽ di chuyển khi phát hiện đường màu đen!")

## 🚀 Bắt đầu chế độ theo đường

**CẢNH BÁO**: Đảm bảo robot được đặt trên đường/track và có đủ không gian trước khi chạy!

In [ ]:
    # Test hàm điều khiển computer vision một lần
    print("🧪 Test hàm điều khiển computer vision với frame hiện tại...")
    execute_cv({'new': camera.value})
    print("✓ Test thành công!")

    # Bắt đầu chế độ theo đường với computer vision
    print("🚀 Bắt đầu chế độ theo đường với Computer Vision...")
    print("⚠️ ROBOT SẼ BẮT ĐẦU DI CHUYỂN KHI PHÁT HIỆN ĐƯỜNG ĐEN!")

    camera.observe(execute_cv, names='value')
    print("✅ Chế độ theo đường Computer Vision đã được kích hoạt!")

## 🛑 Dừng robot

Chạy cell dưới đây để dừng robot và ngắt kết nối camera:

In [ ]:
# Dừng chế độ theo đường computer vision
print("🛑 Đang dừng robot...")

# Ngắt kết nối callback
camera.unobserve(execute_cv, names='value')

# Chờ một chút để xử lý frame cuối cùng
time.sleep(0.1)

# Dừng robot hoàn toàn
robot.stop()

print("✅ Robot đã dừng!")

In [ ]:
# Giải phóng tài nguyên camera
print("🔌 Đang đóng kết nối camera...")
camera.stop()
print("✅ Camera đã được đóng - có thể sử dụng trong notebook khác!")

# ? Computer Vision Line Following - Detect Đường Đen

## Notebook này sử dụng OpenCV để phát hiện đường màu đen

### ? Cách hoạt động:
1. **Chuyển đổi màu**: BGR → Grayscale
2. **Blur**: Giảm noise bằng Gaussian blur  
3. **Threshold**: Tách vùng đen (đường) khỏi nền
4. **Contour Detection**: Tìm đường viền của vùng đen
5. **Center Calculation**: Tính toán tâm của đường
6. **PID Control**: Điều khiển robot theo tâm đường

### 📋 Các bước sử dụng:
1. **Cell 1**: Import thư viện OpenCV
2. **Cell 2**: Khởi tạo camera, robot và hiển thị hình ảnh
3. **Cell 3**: Định nghĩa hàm computer vision
4. **Cell 4**: Test với frame hiện tại
5. **Cell 5-6**: Tạo slider điều khiển
7. **Cell 7**: Hàm điều khiển chính
8. **Cell 8**: Bắt đầu chế độ theo đường
9. **Cell 9-10**: Dừng robot và đóng camera

### ⚙️ Điều chỉnh thông số:

**Điều khiển Robot:**
- **Speed Gain**: Tốc độ robot (0.1-0.3)
- **Steering Gain**: Độ mạnh rẽ (0.3-1.0) 
- **Steering kD**: Giảm dao động (0.01-0.1)
- **Steering Bias**: Hiệu chỉnh lệch (-0.1 đến 0.1)

**Computer Vision:**
- **Black Threshold**: Ngưỡng phát hiện màu đen (30-120)
- **Min Area**: Diện tích tối thiểu của đường (50-500)

### 🎯 Tips để có kết quả tốt:
1. **Đường màu đen rõ ràng**: Sử dụng băng keo đen trên nền sáng
2. **Ánh sáng đều**: Tránh bóng đổ và phản quang
3. **Điều chỉnh threshold**: Thay đổi "Black Threshold" để phát hiện tốt hơn
4. **Bắt đầu chậm**: Speed Gain = 0.1-0.15
5. **Quan sát hình debug**: Kiểm tra vùng đen được phát hiện

### 🔍 Debug:
- **Hình trái**: Camera gốc
- **Hình phải**: Hình đã xử lý (trắng = đường đen phát hiện được)
- **Đường xanh**: Trung tâm camera
- **Điểm đỏ**: Tâm đường phát hiện được
- **Đường xanh dương**: Hướng steering

### 🚨 An toàn:
- Robot chỉ di chuyển khi **confidence > 0.1**
- Tự động dừng khi không phát hiện được đường
- Luôn có không gian an toàn xung quanh

**Chúc bạn thành công với Computer Vision Line Following! 🤖?**